In [171]:
import cv2
import numpy as np
import os

In [172]:
# show img for testing
def show_image_by_OpenCV(img):
    cv2.imshow('My Image', img )
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [173]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

In [ ]:
def detect_Harris_corner(images):
    # parameters
    KERNEL_SIZE = 5
    THRESHOLD = 4000
    K = 0.04

    #  1. Compute x and y derivatives of images.
    # rgb to grayscale
    Gray_imgs = [cv2.cvtColor(i, cv2.COLOR_RGB2GRAY) for i in images]
    # gaussion : to do small smooth
    Gau_imgs = [cv2.GaussianBlur(i, (3,3),3) for i in Gray_imgs]
    # gradient
    I_x = [np.gradient(i, axis=1) for i in Gau_imgs]
    I_y = [np.gradient(i, axis=0) for i in Gau_imgs]


    # 2. Compute products of derivates at each pixel.
    I_xx = [i*i for i in I_x]
    I_yy = [i*i for i in I_y]
    I_xy = [x*y for x,y in zip(I_x, I_y)]

    # 3. Compute the sums of the products of derivates at each pixel.
    # weight : use gaussian
    S_xx = [cv2.GaussianBlur(i, (KERNEL_SIZE,KERNEL_SIZE),KERNEL_SIZE) for i in I_xx]
    S_yy = [cv2.GaussianBlur(i, (KERNEL_SIZE,KERNEL_SIZE),KERNEL_SIZE) for i in I_yy]
    S_xy = [cv2.GaussianBlur(i, (KERNEL_SIZE,KERNEL_SIZE),KERNEL_SIZE) for i in I_xy]
    
    # 4. Define the matrix M.
    # | S_xx  S_xy |
    # | S_xy  S_yy |

    # 5. Compute the response of the detector at each pixel
    detM = [x * y - xy * xy for x, y, xy in zip(S_xx, S_yy, S_xy)]
    traceM = [x + y for x, y in zip(S_xx, S_yy)]
    R = [det_m - K * (trace_m * trace_m) for det_m, trace_m in zip(detM, traceM)]

    # 6. Threshold on value of R
    for i, ele in enumerate(R):
        ele[ele < THRESHOLD] = 0
        ele[ele > 0] = 255
        R[i] = ele
    
    # return images
    return R
        

In [174]:
def detect_Harris_corner(image):
    # parameters
    KERNEL_SIZE = 5
    THRESHOLD = 4000
    K = 0.04

    #  1. Compute x and y derivatives of image.
    # rgb to grayscale
    Gray_img = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) 
    # gaussion : to do small smooth
    Gau_img = cv2.GaussianBlur(Gray_img, (3,3),3)
    # gradient
    I_x = np.gradient(Gau_img, axis=1) 
    I_y = np.gradient(Gau_img, axis=0) 


    # 2. Compute products of derivates at each pixel.
    I_xx = I_x * I_x
    I_yy = I_y * I_y
    I_xy = I_x * I_y

    # 3. Compute the sums of the products of derivates at each pixel.
    # weight : use gaussian
    S_xx = cv2.GaussianBlur(I_xx, (KERNEL_SIZE,KERNEL_SIZE),KERNEL_SIZE)
    S_yy = cv2.GaussianBlur(I_yy, (KERNEL_SIZE,KERNEL_SIZE),KERNEL_SIZE)
    S_xy = cv2.GaussianBlur(I_xy, (KERNEL_SIZE,KERNEL_SIZE),KERNEL_SIZE)
    
    # 4. Define the matrix M.
    # | S_xx  S_xy |
    # | S_xy  S_yy |

    # 5. Compute the response of the detector at each pixel
    detM = S_xx * S_yy - S_xy * S_xy
    traceM = S_xx + S_yy
    R = detM - K * (traceM * traceM)

    # 6. Threshold on value of R
    R[R < THRESHOLD] = 0
    R[R > 0] = 255
    
    # return images
    return R
        

In [175]:
a = load_images_from_folder('img/csie')
R = detect_Harris_corner(a[0])

In [176]:
show_image_by_OpenCV(R)